- 4 часа. Код обучения логрега сформировал,
  - добавив разделение больших описаний на абзацы (качество улучшилось),
  - опционально учет дисбаланса (так в целом классы находит лучше average_precision_score, малые в первую очередь, но чаще классифицирует текст как атаку, когда в реальности ее нет, из-за чего метрика logloss ухудшается),
  - к прежнему отбору признаков по logreg + r1 штраф добавил выбор признаков для каждого класса по куммулятивному вкладу в feature_importance с заданым параметром (поэтом улучшили фильтрацию, перенеся от жесткого филтра частот встречаемости в векторизаторе до ослабления фильтра и отборе на этапе классификации)
  - проверил ухудшает ли стемминг качество для классического алгоритма (без качество хуже). Ранее показывал для Байеса, но там качество было хуже, интересно, что count vectorizer показал лучше качество для итогового логрега.
    
- 3 часа. В исходный код отчета добавил оптимизацию порогов и f1 по классам, матрицу неправильных классификаций по классам

- 8 часов. Построил сверточную сеть для оценки качества и вытянул эмбеддинг для определения близких описаний.
   - Нашел дубли в tram
   - Тексты с кучей мусорных описаний - data/artifacts/tram_rubbish.csv
   - При разбиении описаний, краткие предложения из-за списков (https://attack.mitre.org/techniques/T1110/001/)
   - x_mitre_deprecated флаг в описаниях процедур, из-за которых, к атаке привязываются дубли процедур
   - нашел идентичные описания процедур, хотя техники разные (data/artifacts).
   - убрал скрытые дубли, у которых отличия только в паре слов или символов сохранил в data/artifacts/shadow_duples.csv
- 4 часа. добавил бинарный классификатор на определение атаки, а затем определение самой атаки нет улучшений. Затем добавил в модель как фичу и получил лучше результаты


# Problems

## Выводы

<div class='alert alert-info'>
    - для тактики impact есть характерные сочетания:
        - /all /quiet	0.113068
        - denial servic	0.109864
        - disk structur	0.094003
        - stop servic	0.065820
        - shadow copi	0.058831

        - shadow
        - overwrit

</div>

<div class='alert alert-info'>
    - сочетание reveal for имеет хорошую распознавательную способность для тактики reconnaissance
</div>

## tram

### shadow duples

In [154]:
s1 = 'Inhibit System Recovery T1490'
s2 = 'Umbrella, Cisco’s secure internet gateway (SIG), blocks users from connecting to malicious domains'
# строки немного отличающиеся
data.loc[data.sentence.str.contains(s), 'sentence'].iloc[0],data.loc[data.sentence.str.contains(s), 'sentence'].iloc[1]

('Inhibit System Recovery T1490 CORTEX XSOAR Deploy XSOAR Playbook – Ransomware Manual for incident response. ',
 'Inhibit System Recovery T1490 CORTEX XSOAR Deploy XSOAR Playbook - Ransomware Manual for incident response. ')

## texts like tactics

In [ ]:
data = pd.read_csv(conf['feat_gen']['data_fn'])

data['target'] = data['target'].map(lambda x: eval(x))

data['labels'] = data['labels'].map(lambda x: eval(x))

In [8]:
with pd.option_context('display.max_colwidth', 200):
    display(data[(data['sentence'].str.contains('versary'))&(data['labels'].str.len()==0)])

,sentence,labels,url,par_name,is_proc,origin_labels,prep_text,target,split,threat_words
16868,"Before March 2020, the NetWalker ransom note indicated how to contact the adversary directly using anonymous email account services with random names (such as kkeessnnkkaa@cock.li and hhaaxxhhaaxx...",[],NaN,NaN,NaN,[],befor march 2020 the netwalk ransom note indic how contact the adversari direct use anonym email account servic with random name such cock.li and tuta.io figur,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",ts,[]
17720,This activity is tracked by CrowdStrike as the StellarParticle campaign and is associated with the COZY BEAR adversary group.,[],NaN,NaN,NaN,[],this activ track the campaign and associ with the cozi bear adversari group,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",tr,[]
17722,"These techniques include: Credential hopping for obscuring lateral movement Office 365 (O365) Service Principal and Application hijacking, impersonation and manipulation Stealing browser cookies f...",[],NaN,NaN,NaN,[],these techniqu includ credenti hop for obscur later movement offic 365 o365 servic princip and hijack and steal browser cooki for bypass use the implant and the linux variant goldmax malwar crede...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",tr,[]
17723,"This has been advantageous to CrowdStrike incident responders in that, through investigating victim O365 environments, they could gain an accurate accounting of time, account and source IP address...",[],NaN,NaN,NaN,[],this has been incid respond that through victim o365 they could gain accur account time account and sourc address adversari the o365 tenant,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",tr,[]
17818,"Due to the subdirectories that are named after the FQDNs for victim domains, CrowdStrike assesses with moderate confidence that the scripts represent an AD enumeration tool used by the adversary.",[],NaN,NaN,NaN,[],due the that are name after the fqdns for victim domain assess with moder confid that the script repres tool use the adversari,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",tr,[]
...,...,...,...,...,...,...,...,...,...,...
29198,"Combining Hexray’s Lumina and BinDiff shows that the Standard Bot contains functionality relating to web injects, adversary in the middle (AiTM) and backconnect capabilities that do not exist with...",[],NaN,NaN,NaN,[],combin hexray lumina and bindiff show that the standard bot contain relat web inject adversari the middl aitm and that not exist within the fork variant,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",val,[]
30849,"If NTLM must be enabled, enable Extended Protection for Authentication (EPA) to prevent some NTLM-relay attacks, and implement SMB signing to prevent certain adversary-in-the-middle and pass-the-h...",[],NaN,NaN,NaN,[],ntlm must enabl enabl extend protect for epa prevent some ntlm-relay attack and implement smb sign prevent certain and attack cpg 3.4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",ts,[]
32824,"Its likely that this discovery technique forms an extension of T1033 – System Owner/User Discovery, where the threat actor was leveraging this data source to understand the account pattern of life...",[],NaN,NaN,NaN,[],it like that this discoveri techniqu form extens t1033 system owner/us discoveri where the threat actor was leverag this data sourc understand the account pattern life ani indic from compromis an...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",val,[]
33971,What is Cobalt Strike? Cobalt Strike is marketed as “Software for Adversary Simulations and Red Team Operations.”,[],NaN,NaN,NaN,[],what cobalt strike cobalt strike market softwar for adversari and red team oper,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",tr,[]
